# Thrust vectored AUV

## Extremeley similar to thrust vector rocket landing (a la moda di SpaceX)
![](rocket.png)
![](equations.png)

In [1]:
from sympy import *
init_printing()

In [2]:
# state
x, y, vx, vy, theta, omega = symbols('x y vx vy theta omega', real=True)
p = Matrix([x, y])
v = Matrix([vx, vy])
s = Matrix([p, v, [theta], [omega]])
s.T

[x  y  vx  vy  θ  ω]

In [3]:
# costate variables
lx, ly, lvx, lvy, ltheta, lomega = symbols('lambda_x lambda_y lambda_v_x lambda_v_y lambda_theta lambda_omega', real=True)
lp = Matrix([lx, ly])
lv = Matrix([lvx, lvy])
l = Matrix([lp, lv, [ltheta], [lomega]])
l.T

[λₓ  λ_y  λᵥ ₓ  λ_v_y  λ_θ  λ_ω]

In [4]:
# params: gravity, max thrust, mass, half length of auv
g, T, m, length, alpha, rho, cd, A = symbols('g T m L alpha rho C_d A', real=True, positive=True)
params = Matrix([g, T, m, length, alpha, rho, cd, A])
params.T

[g  T  m  L  α  ρ  C_d  A]

In [5]:
# actions
u = symbols('u', real=True, positive=True)
phi = symbols('phi', real=True)
actions = Matrix([u, phi])
actions.T

[u  φ]

In [6]:
# thrust vector tilt wrt global frame
t = Matrix([cos(theta + phi), sin(theta + phi)])

# rocket orientation wrt horizontal
itheta = Matrix([cos(theta), sin(theta)])

# perpendicular direction to orientation
itau = Matrix([sin(theta), -cos(theta)])

In [7]:
# dynamics
dr = v
dv = T*u*t + Matrix([0, -g]) + Rational(1, 2)*rho*sqrt(sum([vec**2 for vec in v]))*v*cd*A
dtheta = omega
domega = (12/m/length**2) * T*u*length/2 * t.dot(itau)
ds = Matrix([dr, dv, [dtheta], [domega]])
ds

⎡                      vx                      ⎤
⎢                                              ⎥
⎢                      vy                      ⎥
⎢                                              ⎥
⎢                ___________                   ⎥
⎢               ╱   2     2                    ⎥
⎢  A⋅C_d⋅ρ⋅vx⋅╲╱  vx  + vy                     ⎥
⎢  ───────────────────────── + T⋅u⋅cos(φ + θ)  ⎥
⎢              2                               ⎥
⎢                                              ⎥
⎢              ___________                     ⎥
⎢             ╱   2     2                      ⎥
⎢A⋅C_d⋅ρ⋅vy⋅╲╱  vx  + vy                       ⎥
⎢───────────────────────── + T⋅u⋅sin(φ + θ) - g⎥
⎢            2                                 ⎥
⎢                                              ⎥
⎢                      ω                       ⎥
⎢                                              ⎥
⎢6⋅T⋅u⋅(sin(θ)⋅cos(φ + θ) - sin(φ + θ)⋅cos(θ)) ⎥
⎢───────────────────────────────────────────── ⎥
⎣                   

In [8]:
# dynamic Lagrangian to minimise thrust usage
L = (1- alpha)*u**2 + alpha*u
L

       2         
α⋅u + u ⋅(-α + 1)

In [9]:
# Hamiltonian
H = l.dot(ds) + L
H

                   ⎛              ___________                 ⎞         ⎛     
                   ⎜             ╱   2     2                  ⎟         ⎜     
                   ⎜A⋅C_d⋅ρ⋅vx⋅╲╱  vx  + vy                   ⎟         ⎜A⋅C_d
α⋅u + λ_θ⋅ω + λᵥ ₓ⋅⎜───────────────────────── + T⋅u⋅cos(φ + θ)⎟ + λ_v_y⋅⎜─────
                   ⎝            2                             ⎠         ⎝     

         ___________                     ⎞                                    
        ╱   2     2                      ⎟                                    
⋅ρ⋅vy⋅╲╱  vx  + vy                       ⎟                     2            6⋅
──────────────────── + T⋅u⋅sin(φ + θ) - g⎟ + λₓ⋅vx + λ_y⋅vy + u ⋅(-α + 1) + ──
       2                                 ⎠                                    

                                               
                                               
T⋅λ_ω⋅u⋅(sin(θ)⋅cos(φ + θ) - sin(φ + θ)⋅cos(θ))
───────────────────────────────────────────────
                

In [10]:
# optimal thrust direction
laux = (lv + lomega*12/length**2*itau)
lauxnorm = sqrt(sum([laux[i]**2 for i in range(len(laux))]))
lauxdir = laux/lauxnorm
tstar = -lauxdir
tstar = simplify(tstar)
tstar

⎡                  ⎛ 2                     ⎞                   ⎤
⎢                 -⎝L ⋅λᵥ ₓ + 12⋅λ_ω⋅sin(θ)⎠                   ⎥
⎢──────────────────────────────────────────────────────────────⎥
⎢    __________________________________________________________⎥
⎢   ╱                          2                             2 ⎥
⎢  ╱  ⎛ 2                     ⎞    ⎛ 2                      ⎞  ⎥
⎢╲╱   ⎝L ⋅λᵥ ₓ + 12⋅λ_ω⋅sin(θ)⎠  + ⎝L ⋅λ_v_y - 12⋅λ_ω⋅cos(θ)⎠  ⎥
⎢                                                              ⎥
⎢                     2                                        ⎥
⎢                  - L ⋅λ_v_y + 12⋅λ_ω⋅cos(θ)                  ⎥
⎢──────────────────────────────────────────────────────────────⎥
⎢    __________________________________________________________⎥
⎢   ╱                          2                             2 ⎥
⎢  ╱  ⎛ 2                     ⎞    ⎛ 2                      ⎞  ⎥
⎣╲╱   ⎝L ⋅λᵥ ₓ + 12⋅λ_ω⋅sin(θ)⎠  + ⎝L ⋅λ_v_y - 12⋅λ_ω⋅cos(θ)⎠  ⎦

In [11]:
# new Hamiltonian with optimal thrust direction
drs = v
dvs = T*u*t + Matrix([0, -g]) + Rational(1, 2)*rho*sqrt(sum([vec**2 for vec in v]))*v*cd*A
dthetas = omega
domegas = 12*T*u/m/length**2 * t.dot(itau)
dss = Matrix([dr, dv, [dtheta], [domega]])
Hstar = l.dot(dss) + L

In [12]:
# optimal thrust magnitude (unbounded)
ustar = solve(Hstar.diff(u), u)[0]
ustar = simplify(ustar)
ustar

L⋅T⋅λᵥ ₓ⋅m⋅cos(φ + θ) + L⋅T⋅λ_v_y⋅m⋅sin(φ + θ) + L⋅α⋅m - 6⋅T⋅λ_ω⋅sin(φ)
───────────────────────────────────────────────────────────────────────
                             2⋅L⋅m⋅(α - 1)                             

In [13]:
# costate equations of motion
dl = -Matrix([H.diff(var) for var in s])

# fullstate
dfs = Matrix([ds, dl])
dfs = simplify(dfs)
dfs

⎡                                    vx                                     ⎤
⎢                                                                           ⎥
⎢                                    vy                                     ⎥
⎢                                                                           ⎥
⎢                              ___________                                  ⎥
⎢                             ╱   2     2                                   ⎥
⎢                A⋅C_d⋅ρ⋅vx⋅╲╱  vx  + vy                                    ⎥
⎢                ───────────────────────── + T⋅u⋅cos(φ + θ)                 ⎥
⎢                            2                                              ⎥
⎢                                                                           ⎥
⎢                            ___________                                    ⎥
⎢                           ╱   2     2                                     ⎥
⎢              A⋅C_d⋅ρ⋅vy⋅╲╱  vx  + vy                          

# Code

In [17]:
ustar.subs()

L⋅T⋅λᵥ ₓ⋅m⋅cos(φ + θ) + L⋅T⋅λ_v_y⋅m⋅sin(φ + θ) + L⋅α⋅m - 6⋅T⋅λ_ω⋅sin(φ)
───────────────────────────────────────────────────────────────────────
                             2⋅L⋅m⋅(α - 1)                             

In [ ]:
pprint(dfs)

In [ ]:
dfs
